# 20 正则化

## Reduce Overfitting
   * More data
   * Constraint model complexity
        * <font face="黑体" color=green size=3>shallow</font>
        * <font face="黑体" color=green size=3>regularization</font>
   * Dropout
   * Data argumentation
   * Early stopping

### Regularization=weight decay

#### L1-regularization
$$J(\theta) = -\frac {1}{m} \sum_{i=1}^{m}[y_iln \hat{y_i}+(1-y_i)ln(1-\hat{y_i})]+ \lambda \sum_{i=1}^n \left|{\theta _i}\right|$$
#### L2-regularization
$$J(W; X,y)+\frac{1}{2} \lambda \cdot \left||W\right||^2$$

### L2-regulation
```python
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.01)    # add L2-regulation
criteon = nn.CrossEntropyLoss().to(device)
```

### L1-regularization
```python
regularization_loss = 0
for param in model.parameters():
    regularization_loss += torch.sum(torch.abs(param))
    
classify_loss = criteon(logits, target)
loss = classify_loss + 0.01 * regularization_loss    # lambda=0.01

optimizer.zero_grad()
loss.backward()
optimizer.step()
```

# momentum and learning rate decay

### tricks
* momentum
* learning rate decay

### momentum在pytorch中的实现：
```python
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)
```

Adam中已经有momentum。

### learning rate decay在pytorch中的实现：

scheme 1:  

```python
scheduler = ReduceLROnPlateau(optimizer, 'min')

for epoch in xange(args.start_epoch, args.epochs):
    train(train_loader, model, criterion, optimizer, epoch)
    result_avg, loss_val = validate(val_loader, model, criterion, epoch)
    scheduler.step(loss_val)
```

sheme2:
```python
# Assuming optimizer uses lr = 0.05 for all groups
# lr = 0.05     if epoch < 30
# lr = 0.005     if 30 <= epoch < 60
# lr = 0.0005    if 60 <= epoch < 90

scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(100):
    scheduler.step()
    train(...)
    validate(...)
```

# early stop and dropout

### Tricks
* Early stopping
* Dropout
* Stochastic gradient descent

### Early Stopping
<img src='./images/24.png' style="zoom:50%">

### Dropout
<img src='./images/25.png' style="zoom:50%">   
<img src='./images/26.png' style="zoom:50%"> 

```python
net_dropped = torch.nn.Sequential(
    torch.nn.Linear(784, 200)
    torch.nn.Dropout(0.5)             # drop 50% of the neuron
    torch.nn.ReLU()
    torch.nn.Linear(200, 200)
    torch.nn.Dropout(0.5)             # drop 50% of the neuron
    torch.nn.ReLU()
    torch.nn.Linear(200, 10)
```

### Clarification

torch.nn.Dropout(p=dropout_prob)   =1 断  
tf.nn.dropout(keep_prob)        =1保持

### Behavior between train and test

```python
for epoch in range(epochs):
    
    #train
    net_dropped.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        ...
    net_dropped.eval()          # 在validation的时候一定要取消dropout，这样可以提高结果。
    test_loss = 0
    correct = 0
    for data, target in best_loader:
        ...
```

### Stochastic Gradient Descent

###### Stochastic

###### Deterministic